<img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/horizontal-primary-light.png" alt="he-black-box" width="600"/>


# Homomorphic Encryption using Duet: Data Owner
## Tutorial 1: Logistic Regression over Encrypted Data



Welcome!
This tutorial will show you how to train and evaluate a Logistic Regression using Duet and TenSEAL. This notebook shows the Data Scientist view on the operations.


We recommend going through Tutorial 0 before trying this one.

## Setup

All modules are imported here, make sure everything is installed by running the cell below.

In [ ]:
import syft as sy
import tenseal as ts
import torch
import pandas as pd
import random
import numpy as np

import pytest
from time import time
import matplotlib.pyplot as plt

sy.load("tenseal")


### Join the Duet

In [ ]:
duet = sy.join_duet(loopback=True)

### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 1 : Now STOP and run the Data Owner notebook until the next checkpoint.

### Define The Model

Here we define the models used by the Data Scientist.

 - __LR__ is a simple logistic regression for plain data.
 - __EncryptedLR__ adapts the forward method to the API exposed by the encrypted ciphertexts.

In [ ]:
class LR(torch.nn.Module):

    def __init__(self, n_features):
        super(LR, self).__init__()
        self.lr = torch.nn.Linear(n_features, 1)
        
    def forward(self, x):
        out = torch.sigmoid(self.lr(x))
        return out
    
class EncryptedLR:
    def __init__(self, torch_lr):
        # TenSEAL processes lists and not torch tensors
        # so we take out parameters from the PyTorch model
        self.weight = torch_lr.lr.weight.data.tolist()[0]
        self.bias = torch_lr.lr.bias.data.tolist()
        
    def forward(self, enc_x):
        # We don't need to perform sigmoid as this model
        # will only be used for evaluation, and the label
        # can be deduced without applying sigmoid
        enc_out = enc_x.dot(self.weight) + self.bias
        return enc_out
    
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

### Request the training data

We request access to the training data.

In [ ]:
duet.store.pandas

In [ ]:
x_train_ptr = duet.store["x_train"]
y_train_ptr = duet.store["y_train"]
x_train_ptr.request(reason="I would like to get the training data")
y_train_ptr.request(reason="I would like to get the training labels")


In [ ]:
duet.requests.pandas

### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 2 : Now STOP and run the Data Owner notebook until the next checkpoint.

### Train the logistic regression

Now we train our Logistic regression with the data from the DO instance.

In [ ]:
x_train = x_train_ptr.get(delete_obj=False)
y_train = y_train_ptr.get(delete_obj=False)

n_features = x_train.shape[1]
model = LR(n_features)

# use gradient descent with a learning_rate=1
optim = torch.optim.SGD(model.parameters(), lr=1)

# use Binary Cross Entropy Loss
criterion = torch.nn.BCELoss()

In [ ]:
EPOCHS = 20

def train(model, optim, criterion, x, y, epochs=EPOCHS):
    for e in range(1, epochs + 1):
        optim.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optim.step()
        print(f"Loss at epoch {e}: {loss.data}")
    return model

model = train(model, optim, criterion, x_train, y_train)

### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 3 : Now STOP and run the Data Owner notebook until the next checkpoint.

### Request the encrypted tensors

We create the model variant that cand handle encrypted tensors.

In [ ]:
eelr = EncryptedLR(model)

### Check the Duet store

Now we can see all evaluating objects and get pointers to them.


__Note__: we cannot get these without permission.

In [ ]:
duet.store.pandas

### Get pointers to the encrypted vectors and request permission to work on them.

In [ ]:
ctx_ptr = duet.store["context"]
enc_x_test_ptr = duet.store["enc_x_test"]

ctx_ptr.request(reason="I would like to get the context")
enc_x_test_ptr.request(reason="I would like to get encrypted test set")

### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 4 : Now STOP and run the Data Owner notebook until the next checkpoint.

### Encrypted evaluation

In [ ]:
def encrypted_evaluation(model, enc_x_test):
    results = []
    
    correct = 0
    for enc_x in enc_x_test:
        # encrypted evaluation
        results.append(model(enc_x))
    return results

enc_x_test = enc_x_test_ptr.get(delete_obj=False)
ctx = ctx_ptr.get(delete_obj=False)
for t in enc_x_test:
    t.link_context(ctx)

eval_res = encrypted_evaluation(eelr, enc_x_test)

In [ ]:
result_eval_ptr = sy.lib.python.List(eval_res).send(duet, pointable=True, tags=["result_eval"])

print(duet.store)

### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 5 : Now STOP and run the Data Owner notebook until the next checkpoint.

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft and TenSEAL on GitHub

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)
- [Star TenSEAL](https://github.com/OpenMined/TenSEAL)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org). #lib_tenseal and #code_tenseal are the main channels for the TenSEAL project.

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)